In [1]:
import json
import sys

sys.path.append('..')
from tools.TextProcessing import sent_lemmatize, build_word_tree

In [ ]:
# Process keyword file (20 seconds)

filtered_words = set(['can', 'it', 'work', 'in', 'parts', 'is', 'its', 'or', 'and', 'a','b','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', ''])

stable_kw = []
unstable_kw = []
with open('../data/raw_data/keyword.csv', 'r') as f_in:
    for line in f_in:
        kw = line.split(',')[0]
        if '- ' in kw:
            continue
        splited = kw.replace('-', ' - ')
        reformed = ' '.join(sent_lemmatize(splited))
        if reformed in filtered_words:
            continue
        if reformed == splited:
            stable_kw.append(kw)
        else:
            unstable_kw.append('%s\t%s' % (kw, reformed))

with open('../data/corpus/keyword_f.txt', 'w') as f_out:
    f_out.write('\n'.join(stable_kw))

with open('../data/temp/unstable_keyword.txt', 'w') as f_out:
    f_out.write('\n'.join(unstable_kw))

In [ ]:
# Check stable keywords
'-' in stable_kw

In [13]:
# Generate word tree (25 seconds)
build_word_tree('../data/corpus/keyword_f.txt', '../data/corpus/wordtree.json', '../data/corpus/entity.txt')

Building word tree is accomplished with 12699 words added


In [ ]:
# Extract sentence from small corpus (2 minute)
j = json.load(open('../data/raw_data/small_arxiv.json', 'r'))
sents = [i['abstract'].replace('\n', ' ').strip() for i in j]
with open('../data/corpus/pre_small_sent.txt', 'w') as f_out:
    f_out.write('\n'.join(sents))

!sed 's/\$//g;s/\\/ /g;s/---*/, /g;s/([^)]*)//g;s/{[^)]*}//g;s/-/ - /g' ../data/corpus/pre_small_sent.txt | tr -s [:space:] | tr '[:upper:]' '[:lower:]' > ../data/corpus/small_sent.txt

!python ../py/sent_tokenize.py ../data/corpus/small_sent.txt ../data/corpus/small_sent.txt

!rm ../data/corpus/pre_small_sent.txt

In [ ]:
# Extract sentence from 1st_sents_new.json
file_dir = '../data/raw_data/1st-sents-new.json'

j = json.load(open(file_dir, 'r'))
sents = [i['sentence'].strip() for i in j.values()]
with open('../data/corpus/pre_1st_sent.txt', 'w') as f_out:
    f_out.write('\n'.join(sents))

!sed 's/\$//g;s/\\/ /g;s/---*/, /g;s/([^)]*)//g;s/{[^)]*}//g;s/-/ - /g' ../data/corpus/pre_1st_sent.txt | tr -s [:space:] | tr '[:upper:]' '[:lower:]' > ../data/corpus/1st_sent.txt

!rm ../data/corpus/pre_1st_sent.txt

In [ ]:
# Generate Reformed sentences (3 minutes)
# To run the code in the backend, use the gen_reform.py in the "py" folder

## Keyword process using terms-cs-cfl-epoch200.txt and 1st-sents-new.json

In [2]:
import json
import csv

filtered_words = set(['can', 'it', 'work', 'in', 'parts', 'is', 'its', 'or', 'and', 'a','b','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', ''])

wiki_kw_dict = json.load(open('../data/corpus/1st-sents-lowercase.json', 'r'))

In [19]:
stable_kw = []
unstable_kw = []
with open('../data/raw_data/terms-cs-cfl-epoch200.txt', 'r') as f_in:
    r = csv.reader(f_in, delimiter='\t')
    candidate_kw_list = [item[0] for item in r if item[0] in wiki_kw_dict]
    print('convolutional neural network' in candidate_kw_list)
    for kw in candidate_kw_list:
        if '- ' in kw:
            continue
        splited = kw.replace('-', ' - ')
        reformed = ' '.join(sent_lemmatize(splited))
        if reformed in filtered_words:
            continue
        if reformed == splited:
            stable_kw.append(kw)
        else:
            unstable_kw.append('%s\t%s' % (kw, reformed))

True


In [12]:
wiki_kw_dict['python']

{'sentence': 'python was a cold war contingency plan of the british government for the continuity of government in the event of nuclear war .',
 'entity': {'cold war': 'cold war',
  'british government': 'government of the united kingdom',
  'continuity of government': 'continuity of government',
  'nuclear war': 'nuclear warfare'}}

In [20]:
'convolutional neural network' in candidate_kw_list

True

In [21]:
with open('../data/corpus/keyword_f.txt', 'w') as f_out:
    f_out.write('\n'.join(stable_kw))